<a href="https://colab.research.google.com/github/dinagalevska/NLP/blob/master/NLP_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install evaluate
!pip install datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/NLP')

In [4]:
import pandas as pd
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from evaluate import load
from seq2seq import create_transformers_train_data, decode_with_transformer
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm
import tensorflow as tf

In [ ]:
data = pd.read_csv("hf://datasets/s-nlp/paradetox/train.tsv", sep="\t")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
data

,en_toxic_comment,en_neutral_comment
0,he had steel balls too !,he was brave too!
1,"dude should have been taken to api , he would ...",It would have been good if he went to api. He ...
2,"im not gonna sell the fucking picture , i just...","I'm not gonna sell the picture, i just want to..."
3,the garbage that is being created by cnn and o...,the news that is being created by cnn and othe...
4,the reason they dont exist is because neither ...,The reason they don't exist is because neither...
...,...,...
19739,when they do shit like this .,when they do stuff like this
19740,"but if saying "" fuck that group "" is much more...","but if saying"" that group is bad"" is much more..."
19741,"it hurts how judgemental assholes view them , ...",It hurts how judgemental that people view them...
19742,shit we probably literally blow that up in a w...,We probably litteralky blow that up in a week.


In [ ]:
modern_english_texts = data['en_toxic_comment'].values.tolist()
shakespearean_texts = data['en_neutral_comment'].values.tolist()

In [ ]:
train_en, test_en, train_sh, test_sh = train_test_split(modern_english_texts, shakespearean_texts,
                 test_size=0.2, random_state=0)

#Exercise 1

In [ ]:
t5_model_name = "t5-small"

In [ ]:
t5_model = TFAutoModelForSeq2SeqLM.from_pretrained(t5_model_name)
flan_t5_model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
flan_t5_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you

###T5

In [ ]:
train_set = create_transformers_train_data(train_en, train_sh, t5_tokenizer)


In [ ]:
train_set

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 15795
})

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=t5_tokenizer,
                                       model=t5_model,
                                       return_tensors='tf')

train_set = t5_model.prepare_tf_dataset(train_set,
                                     collate_fn=data_collator)

In [ ]:
train_set

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(8, 10), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(8, 10), dtype=tf.int64, name=None), 'decoder_input_ids': TensorSpec(shape=(8, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(8, None), dtype=tf.int64, name=None))>

In [ ]:
t5_model.compile('adam')

In [ ]:
t5_model.fit(train_set, epochs=3)

Epoch 1/3
1974/1974 [==============================] - 154s 65ms/step - loss: 1.9014
Epoch 2/3
1974/1974 [==============================] - 129s 65ms/step - loss: 1.5684
Epoch 3/3
1974/1974 [==============================] - 131s 66ms/step - loss: 1.3789


In [ ]:
input_example = test_en[0]
gt_example = test_sh[0]

In [ ]:
predicted_example = decode_with_transformer(test_en[0], t5_tokenizer, t5_model)

In [ ]:
print(f'Input: {input_example}\nGT: {gt_example}\nPred: {predicted_example}')

Input: seriously , what the fuck are you going to do with your ar - 92 against an airstrike .
GT: Seriously what are you going to do with your ar-92 against an airstike.
Pred: Seriously, what are you going to do with


In [ ]:
inputs = test_en[:100]
gts = test_sh[:100]
preds = []
for inp in inputs:
  pred = decode_with_transformer(inp, t5_tokenizer, t5_model)
  preds.append(pred)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
for inp, gt, pred in zip(inputs, gts, preds):
  print(f'Input: {inp}\nGT: {gt}\nPred: {pred}\n')

Input: seriously , what the fuck are you going to do with your ar - 92 against an airstrike .
GT: Seriously what are you going to do with your ar-92 against an airstike.
Pred: Seriously, what are you going to do with

Input: fire her ass or she can look for another job
GT: Fire her or she can look for another job
Pred: Fire her or she can look for another job

Input: dnt let tht shit tht happened in yoo room get to yoo head boy
GT: Dont let that incident which happened in that room to effect your head
Pred: tht happened in yoo

Input: who the fuck cares what either of them think ?
GT: who cares what either of them think ?
Pred: Who cares what either of them think?

Input: ? get that shit out of here lol
GT: ? get that out of here lol
Pred: Get that out of here lol

Input: we 've just got back , and now you 've gone and fuckin upset her !
GT: We've just got back, and now you've gone and upset her
Pred: We've just got back and now you

Input: the aclu has been around for almost 21 years 

In [ ]:
bleu_metric = load('bleu')
meteor_metric = load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
bleu_metric.compute(predictions=preds, references=gts)

{'bleu': 0.3043587479861121,
 'precisions': [0.719551282051282,
  0.5820610687022901,
  0.48,
  0.41284403669724773],
 'brevity_penalty': 0.5670500586404044,
 'length_ratio': 0.6380368098159509,
 'translation_length': 624,
 'reference_length': 978}

In [ ]:
meteor_metric.compute(predictions=preds, references=gts)

{'meteor': 0.5419631742949832}

###FLAN-T5.

In [ ]:
train_set = create_transformers_train_data(train_en, train_sh, flan_t5_tokenizer)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=flan_t5_tokenizer,
                                       model=flan_t5_model,
                                       return_tensors='tf')

train_set = flan_t5_model.prepare_tf_dataset(train_set,
                                     collate_fn=data_collator)

In [ ]:
flan_t5_model.compile('adam')

In [ ]:
flan_t5_model.fit(train_set, epochs=3)

Epoch 1/3
1974/1974 [==============================] - 233s 96ms/step - loss: 1.9004
Epoch 2/3
1974/1974 [==============================] - 184s 93ms/step - loss: 1.5066
Epoch 3/3
1974/1974 [==============================] - 190s 96ms/step - loss: 1.2692


In [ ]:
input_example = test_en[0]
gt_example = test_sh[0]

In [ ]:
predicted_example = decode_with_transformer(test_en[0], flan_t5_tokenizer, flan_t5_model)

In [ ]:
print(f'Input: {input_example}\nGT: {gt_example}\nPred: {predicted_example}')

Input: seriously , what the fuck are you going to do with your ar - 92 against an airstrike .
GT: Seriously what are you going to do with your ar-92 against an airstike.
Pred: Seriously, what are you going to do with


In [ ]:
inputs = test_en[:100]
gts = test_sh[:100]
preds = []
for inp in inputs:
  pred = decode_with_transformer(inp, flan_t5_tokenizer, flan_t5_model)
  preds.append(pred)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
for inp, gt, pred in zip(inputs, gts, preds):
  print(f'Input: {inp}\nGT: {gt}\nPred: {pred}\n')

Input: seriously , what the fuck are you going to do with your ar - 92 against an airstrike .
GT: Seriously what are you going to do with your ar-92 against an airstike.
Pred: Seriously, what are you going to do with

Input: fire her ass or she can look for another job
GT: Fire her or she can look for another job
Pred: Fire her or she can look for another job

Input: dnt let tht shit tht happened in yoo room get to yoo head boy
GT: Dont let that incident which happened in that room to effect your head
Pred: I don't want to hear about it

Input: who the fuck cares what either of them think ?
GT: who cares what either of them think ?
Pred: Who cares what either of them think?

Input: ? get that shit out of here lol
GT: ? get that out of here lol
Pred: get that out of here lol

Input: we 've just got back , and now you 've gone and fuckin upset her !
GT: We've just got back, and now you've gone and upset her
Pred: We've just got back and now you

Input: the aclu has been around for almost

In [ ]:
bleu_metric = load('bleu')
meteor_metric = load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
bleu_metric.compute(predictions=preds, references=gts)

{'bleu': 0.30770019855441455,
 'precisions': [0.717628705148206,
  0.5693160813308688,
  0.4683257918552036,
  0.38372093023255816],
 'brevity_penalty': 0.591117166686204,
 'length_ratio': 0.6554192229038854,
 'translation_length': 641,
 'reference_length': 978}

In [ ]:
meteor_metric.compute(predictions=preds, references=gts)

{'meteor': 0.5513829949559432}

#Excercise 2

In [ ]:
data = pd.read_csv("hf://datasets/s-nlp/paradetox/train.tsv", sep="\t")

In [ ]:
modern_english_texts = data['en_toxic_comment'].values.tolist()
shakespearean_texts = data['en_neutral_comment'].values.tolist()

In [ ]:
train_en, test_en, train_sh, test_sh = train_test_split(modern_english_texts, shakespearean_texts, test_size=0.2, random_state=0)

In [ ]:
val_en, test_en, val_sh, test_sh = train_test_split(test_en, test_sh, test_size=0.5, random_state=0)

In [ ]:
flan_t5_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
flan_t5_model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
prompts = [
    "Here is a text: <text>, which is <label1>. Here is a rewrite of the text: <text>, which is <label>. Rewrite the following text: <text> in <label>.",
    "Here is a text: <text>, which is <label1>. Here is a rewrite of the text: <text>, which is not <label1>. Rewrite the following text: <text> in <label>.",
    "Here is a text: <text>, which is not <label2>. Here is a rewrite of the text: <text>, which is <label2>. Rewrite the following text: <text> in <label>."
]

In [ ]:
def create_few_shot_example(n, examples, prompt_template):
    few_shot_example = ""
    for i in range(n):
        text, label = examples[i]
        few_shot_example += prompt_template.replace("<text>", text).replace("<label>", label) + "\n"
    return few_shot_example

#####Evaluation only on test set

In [ ]:
n_values = [1, 2, 3]
test_en_subset = test_en[:100]
gts = test_sh[:100]

bleu_metric = load('bleu')
meteor_metric = load('meteor')

for n in n_values:
    print(f"Testing with n = {n} examples")

    predictions = []
    prompts = [create_few_shot_example(n, list(zip(train_en, train_sh)), prompts[0]).replace("<text>", text) for text in test_en_subset]

    inputs = flan_t5_tokenizer(prompts, padding=True, truncation=True, return_tensors="tf")

    output = flan_t5_model.generate(**inputs, max_new_tokens=50, num_beams=1)  # Limit generation tokens

    predictions = [flan_t5_tokenizer.decode(g, skip_special_tokens=True) for g in output]

    bleu_score = bleu_metric.compute(predictions=predictions, references=gts)
    meteor_score = meteor_metric.compute(predictions=predictions, references=gts)

    print(f"BLEU Score for n={n}: {bleu_score}")
    print(f"METEOR Score for n={n}: {meteor_score}")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Testing with n = 1 examples
BLEU Score for n=1: {'bleu': 0.0, 'precisions': [0.070625, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.6359918200408998, 'translation_length': 1600, 'reference_length': 978}
METEOR Score for n=1: {'meteor': 0.08378203601982769}
Testing with n = 2 examples
BLEU Score for n=2: {'bleu': 0.0, 'precisions': [0.062380952380952384, 0.0015, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 2.147239263803681, 'translation_length': 2100, 'reference_length': 978}
METEOR Score for n=2: {'meteor': 0.10022181515122444}
Testing with n = 3 examples
BLEU Score for n=3: {'bleu': 0.0, 'precisions': [0.04393939393939394, 0.000625, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 3.374233128834356, 'translation_length': 3300, 'reference_length': 978}
METEOR Score for n=3: {'meteor': 0.07280849330555964}


#####Evaluation on train, val, test set like is needed

In [ ]:
n_values = [1, 2, 3]
# test_en_subset = test_en[:100]
# gts = test_sh[:100]

bleu_metric = load('bleu')
meteor_metric = load('meteor')

gts_train = train_sh[:100]
gts_val = val_sh[:100]
gts_test = test_sh[:100]

subsets = {
    "train": (train_en[:100], gts_train),
    "validation": (val_en[:100], gts_val),
    "test": (test_en[:100], gts_test),
}

for subset_name, (inputs, gts) in subsets.items():
    print(f"Evaluating on {subset_name} subset")
    for n in n_values:
        print(f"Testing with n = {n} examples")

        prompts_for_input = [
            create_few_shot_example(n, list(zip(train_en, train_sh)), prompts[0]).replace("<text>", text)
            for text in inputs
        ]

        inputs_encoded = flan_t5_tokenizer(prompts_for_input, padding=True, truncation=True, return_tensors="tf")

        output = flan_t5_model.generate(**inputs_encoded, max_new_tokens=50, num_beams=1)
        predictions = [flan_t5_tokenizer.decode(g, skip_special_tokens=True) for g in output]

        bleu_score = bleu_metric.compute(predictions=predictions, references=gts)
        meteor_score = meteor_metric.compute(predictions=predictions, references=gts)

        print(f"BLEU Score for n={n} on {subset_name}: {bleu_score}")
        print(f"METEOR Score for n={n} on {subset_name}: {meteor_score}")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Evaluating on train subset
Testing with n = 1 examples
BLEU Score for n=1 on train: {'bleu': 0.01589447925462846, 'precisions': [0.0825, 0.010666666666666666, 0.008571428571428572, 0.008461538461538461], 'brevity_penalty': 1.0, 'length_ratio': 1.6080402010050252, 'translation_length': 1600, 'reference_length': 995}
METEOR Score for n=1 on train: {'meteor': 0.09747743848365412}
Testing with n = 2 examples
BLEU Score for n=2 on train: {'bleu': 0.007961568256608667, 'precisions': [0.040625, 0.006, 0.004285714285714286, 0.0038461538461538464], 'brevity_penalty': 1.0, 'length_ratio': 1.6080402010050252, 'translation_length': 1600, 'reference_length': 995}
METEOR Score for n=2 on train: {'meteor': 0.04113731082140964}
Testing with n = 3 examples
BLEU Score for n=3 on train: {'bleu': 0.004667759952962072, 'precisions': [0.0575, 0.004210526315789474, 0.0016666666666666668, 0.001176470588235294], 'brevity_penalty': 1.0, 'length_ratio': 2.0100502512562812, 'translation_length': 2000, 'reference_

#Exercise 3

In [8]:
data = pd.read_csv("hf://datasets/s-nlp/paradetox/train.tsv", sep="\t")
modern_english_texts = data['en_toxic_comment'].values.tolist()
shakespearean_texts = data['en_neutral_comment'].values.tolist()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [39]:
data = data[-500:]
data

,en_toxic_comment,en_neutral_comment
19244,no one implies that you piece of shit .,No one implies that you bad person
19245,you definitely can get herpes viral infection ...,you definitely can get herpes viral infection ...
19246,it 's the fucking iraqi air force .,It's the iraqi air force.
19247,they might just be soon enough if they don 't ...,they might just be soon enough if they don 't ...
19248,shit law is shit law .,Bad law is bad law
...,...,...
19739,when they do shit like this .,when they do stuff like this
19740,"but if saying "" fuck that group "" is much more...","but if saying"" that group is bad"" is much more..."
19741,"it hurts how judgemental assholes view them , ...",It hurts how judgemental that people view them...
19742,shit we probably literally blow that up in a w...,We probably litteralky blow that up in a week.


In [40]:
instruction_transformation = 'Transform the following toxic sentence into a neutral one: '

In [41]:
sentences = [f'{instruction_transformation}{s}' for s in modern_english_texts]
sentences

['Transform the following toxic sentence into a neutral one: he had steel balls too !',
 'Transform the following toxic sentence into a neutral one: dude should have been taken to api , he would be right at home with all the other knuckleheads there',
 'Transform the following toxic sentence into a neutral one: im not gonna sell the fucking picture , i just want to contribute to the fucking article .',
 'Transform the following toxic sentence into a neutral one: the garbage that is being created by cnn and other news agencies is outrageous .',
 'Transform the following toxic sentence into a neutral one: the reason they dont exist is because neither is a pathological liar like trump .',
 'Transform the following toxic sentence into a neutral one: i hope they beat each other silly .',
 'Transform the following toxic sentence into a neutral one: no good bastards that we are and we are unrepentant about it .',
 'Transform the following toxic sentence into a neutral one: stop the coverage a

In [42]:
input_sentences = sentences

In [43]:
output_sentences =  shakespearean_texts

In [44]:
train_en, test_en, train_es, test_es = train_test_split(input_sentences, output_sentences,
                 test_size=0.2, random_state=0)

In [45]:
model_name = 't5-small'

In [46]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [47]:
train_set = create_transformers_train_data(train_en, train_es, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [48]:
train_set

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 15795
})

In [49]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       model=model_name,
                                       return_tensors='tf')

train_set = model.prepare_tf_dataset(train_set,
                                     collate_fn=data_collator)

In [50]:
train_set

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(8, 10), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(8, 10), dtype=tf.int64, name=None)}, TensorSpec(shape=(8, None), dtype=tf.int64, name=None))>

In [51]:
model.compile('adam')

In [52]:
model.fit(train_set, epochs=3)

Epoch 1/3
1974/1974 [==============================] - 161s 64ms/step - loss: 4.0058
Epoch 2/3
1974/1974 [==============================] - 130s 66ms/step - loss: 3.6467
Epoch 3/3
1974/1974 [==============================] - 127s 65ms/step - loss: 3.4212


In [53]:
inputs = test_en[:70]
gts = test_es[:70]

batch_size = 10
preds = []

for i in tqdm(range(0, len(inputs), batch_size)):
    batch_inputs = inputs[i:i + batch_size]
    batch_preds = [decode_with_transformer(inp, tokenizer, model) for inp in batch_inputs]
    preds.extend(batch_preds)

for inp, gt, pred in zip(inputs, gts, preds):
    print(f'Input: {inp}\nGT: {gt}\nPred: {pred}\n')

100%|██████████| 7/7 [01:38<00:00, 14.05s/it]

Input: Transform the following toxic sentence into a neutral one: seriously , what the fuck are you going to do with your ar - 92 against an airstrike .
GT: Seriously what are you going to do with your ar-92 against an airstike.
Pred: I'm gonna be ta

Input: Transform the following toxic sentence into a neutral one: fire her ass or she can look for another job
GT: Fire her or she can look for another job
Pred: I'm gonna be able to

Input: Transform the following toxic sentence into a neutral one: dnt let tht shit tht happened in yoo room get to yoo head boy
GT: Dont let that incident which happened in that room to effect your head
Pred: I'm gonna be ta

Input: Transform the following toxic sentence into a neutral one: who the fuck cares what either of them think ?
GT: who cares what either of them think ?
Pred: I'm gonna be able to

Input: Transform the following toxic sentence into a neutral one: ? get that shit out of here lol
GT: ? get that out of here lol
Pred: I'm gonna be able to

In [54]:
metric = load('bleu')

In [55]:
metric.compute(predictions=preds, references=gts)

{'bleu': 0.0,
 'precisions': [0.04234527687296417,
  0.008438818565400843,
  0.005988023952095809,
  0.0],
 'brevity_penalty': 0.2853406180635688,
 'length_ratio': 0.4436416184971098,
 'translation_length': 307,
 'reference_length': 692}

######за подобри резултати треба повеќе епохи, а и секако целата дата, но многу време се чека, па затоа се вакви резултатите, земам само inputs = test_en[:70]gts = test_es[:70], и само на 3 епохи тренирам

#Exercise 4

In [33]:
data = pd.read_csv("hf://datasets/s-nlp/paradetox/train.tsv", sep="\t")
data = data[:500]
modern_english_texts = data['en_toxic_comment'].values.tolist()
shakespearean_texts = data['en_neutral_comment'].values.tolist()

In [34]:
instruction_transformation = 'Rewrite the following toxic sentence to make it neutral, removing offensive or harmful language: '
instruction_transformation_2 = 'Determine if the following sentence contains offensive language or not. Respond with "toxic" if it is offensive or "neutral" if it is not: '

In [35]:
all_sentences = modern_english_texts + shakespearean_texts

In [36]:
labels = [1] * len(modern_english_texts) + [0] * len(shakespearean_texts)

In [37]:
print(f"Total sentences: {len(all_sentences)}")
print(f"Total labels: {len(labels)}")
print(f"Example sentence: {all_sentences[0]}, Label: {labels[0]}")
print(f"Example sentence: {all_sentences[len(modern_english_texts)]}, Label: {labels[len(modern_english_texts)]}")

Total sentences: 1000
Total labels: 1000
Example sentence: he had steel balls too !, Label: 1
Example sentence: he was brave too!, Label: 0


In [38]:
classification_inputs = [f'{instruction_transformation_2}{s}' for s in all_sentences]
classification_outputs = ['toxic' if label == 1 else 'neutral' for label in labels]

In [39]:
transformation_inputs = [f'{instruction_transformation}{s}' for s in modern_english_texts]
transformation_outputs = shakespearean_texts

In [40]:
train_trans_inputs, test_trans_inputs, train_trans_outputs, test_trans_outputs = train_test_split(
    transformation_inputs, transformation_outputs, test_size=0.2, random_state=42
)

In [41]:
train_class_inputs, test_class_inputs, train_class_outputs, test_class_outputs = train_test_split(
    classification_inputs, classification_outputs, test_size=0.2, random_state=42
)

In [42]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [43]:
def tokenize_data(inputs, outputs, tokenizer, max_length=512):
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length", return_tensors="tf")
    labels = tokenizer(outputs, max_length=max_length, truncation=True, padding="max_length", return_tensors="tf").input_ids
    model_inputs["labels"] = labels
    return model_inputs

In [44]:
train_trans_data = tokenize_data(train_trans_inputs, train_trans_outputs, tokenizer)
test_trans_data = tokenize_data(test_trans_inputs, test_trans_outputs, tokenizer)

In [45]:
train_class_data = tokenize_data(train_class_inputs, train_class_outputs, tokenizer)
test_class_data = tokenize_data(test_class_inputs, test_class_outputs, tokenizer)

In [46]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [47]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [48]:
train_trans_dataset = tf.data.Dataset.from_tensor_slices((dict(train_trans_data), train_trans_data["labels"])).batch(16)
test_trans_dataset = tf.data.Dataset.from_tensor_slices((dict(test_trans_data), test_trans_data["labels"])).batch(16)

In [49]:
train_class_dataset = tf.data.Dataset.from_tensor_slices((dict(train_class_data), train_class_data["labels"])).batch(16)
test_class_dataset = tf.data.Dataset.from_tensor_slices((dict(test_class_data), test_class_data["labels"])).batch(16)

In [50]:
model.compile('adam')

In [51]:
model.fit(train_trans_dataset, validation_data=test_trans_dataset, epochs=1)

25/25 [==============================] - 69s 2s/step - loss: 1.0140 - val_loss: 0.1031


In [52]:
model.fit(train_class_dataset, validation_data=test_class_dataset, epochs=1)

50/50 [==============================] - 79s 2s/step - loss: 0.0484 - val_loss: 0.0095


In [59]:
test_trans_inputs = test_trans_inputs[:50]
test_trans_outputs = test_trans_outputs[:50]

predictions = []
for inp in tqdm(test_trans_inputs, desc="Generating predictions"):
    pred = decode_with_transformer(inp, tokenizer, model)
    if not pred.strip():
        pred = "<EMPTY>"  # deka ne mi e cela data mora vaka za da ne frla error, inaku mnogu vreme trae sekoja kelija soodvetno
    predictions.append(pred)

references = [[ref] for ref in test_trans_outputs]
bleu_metric = load("bleu")
bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU Score for Transformation: {bleu_score['bleu']}")

meteor_metric = load("meteor")
meteor_score = meteor_metric.compute(predictions=predictions, references=test_trans_outputs)
print(f"METEOR Score for Transformation: {meteor_score['meteor']}")

Generating predictions: 100%|██████████| 50/50 [01:22<00:00,  1.65s/it]


BLEU Score for Transformation: 0.0
METEOR Score for Transformation: 0.0


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [60]:
test_class_inputs = test_class_inputs[:50] #isto kako i pogore komentarot
test_class_outputs = test_class_outputs[:50] #isto kako i pogore komentarot

correct_preds = sum([decode_with_transformer(inp, tokenizer, model) == ref for inp, ref in zip(test_class_inputs, test_class_outputs)])
accuracy = correct_preds / len(test_class_outputs)
print(f"Classification Accuracy: {accuracy}")

Classification Accuracy: 0.0


#####probav drug pristap za podobri rezultati, lossot e pomal i beshe pomal na cel data koga probuvav no nadolu sekoja kelija se izvrshuva se povekje vreme, zatoa na kraj se loshi rezultatite bidejki se cheka mnogu vreme ne ja zemam cela data